# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
city_data_df=pd.read_csv("/Users/elliottlevy/Downloads/mycities.csv")
city_data_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Albany,5,US,1606188608,74,42.60,-73.97,35.01,1.01
1,Orange Cove,90,US,1606188940,76,36.62,-119.31,51.01,3.36
2,Vaini,75,TO,1606188689,74,-21.20,-175.20,78.80,10.29
3,Barrow,90,US,1606188690,85,71.29,-156.79,12.20,11.41
4,Tuatapere,93,NZ,1606188704,56,-46.13,167.68,63.00,10.00
...,...,...,...,...,...,...,...,...,...
548,Bereznehuvate,100,UA,1606189306,87,47.31,32.85,38.86,11.16
549,Kuusamo,75,FI,1606189306,92,65.97,29.18,30.20,9.17
550,Luganville,75,VU,1606189306,83,-15.53,167.17,82.40,11.41
551,St. Pete Beach,0,US,1606189306,87,27.73,-82.74,68.00,11.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)
locations_df = city_data_df[["Lat", "Lng"]].astype(float)
humidity_df = city_data_df["Humidity"].astype(float)

In [13]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False, 
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 60) & (city_data_df["Max Temp"] < 90) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 20), :]
ideal_city_df = ideal_city_df.dropna(how='any')
ideal_city_df.reset_index(inplace=True)
del ideal_city_df['index']
ideal_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bella Union,0,UY,1606189193,51,-30.28,-57.60,71.1,8.32
1,Agüimes,0,ES,1606188917,43,27.91,-15.45,73.0,4.70
2,Lebowakgomo,0,ZA,1606189202,82,-24.20,29.50,64.4,8.05
3,Salalah,0,OM,1606188731,65,17.02,54.09,80.6,7.92
4,Praia,0,CV,1606189205,61,14.92,-23.51,77.0,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
hotellist = []

for i in range(len(ideal_city_df)):
    lat = ideal_city_df.loc[i]['Lat']
    lng = ideal_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
ideal_city_df["Hotel Name"] = hotellist
hotel_df = ideal_city_df.dropna(how='any')
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Bella Union,0,UY,1606189193,51,-30.28,-57.60,71.1,8.32,Monte Caseros
1,Agüimes,0,ES,1606188917,43,27.91,-15.45,73.0,4.70,"Ingenio, Las Palmas"
2,Lebowakgomo,0,ZA,1606189202,82,-24.20,29.50,64.4,8.05,Ga-Chuene
3,Salalah,0,OM,1606188731,65,17.02,54.09,80.6,7.92,Salalah
4,Praia,0,CV,1606189205,61,14.92,-23.51,77.0,4.70,Praia


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))